<a href="https://colab.research.google.com/github/germanjke/Deep_Learning_School_MIPT/blob/master/nlp_homeworks/homeworkTextSummarization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization. Homework

Всем привет! Это домашка по суммаризации текста.

На семинаре мы рассмотрели базовые модели для суммаризации текста. Попробуйте теперь улучшить два метода: TextRank и Extractive RNN. Задание достаточно большое и требует хорошую фантазию, тут можно эксперементировать во всю.

Для сдачи заданий надо получить определенное качество по test-у:

- 1 задание: 0.35 BLEU
- 2 задание: 0.35 BLEU

Если ваш подход пробивает это качество – задание считается пройденным. Плюсом будет описание того, почему вы решили использовать то или иное решение. 

Датасет: gazeta.ru

**P.S.** Возможно, в датасете находятся пустые данные. Проверьте эту гипотезу, и если надо, сделайте предобратоку датасета.


`Ноутбук создан на основе семинара Гусева Ильи на кафедре компьютерной лингвистики МФТИ.`

Загрузим датасет и необходимые библиотеки

In [1]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [2]:
!pip install urllib3==1.24.3

In [3]:
!pip install transformers

     |████████████████████████████████| 1.4MB 14.7MB/s 
     |████████████████████████████████| 890kB 53.6MB/s 
     |████████████████████████████████| 2.9MB 65.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=fd394ad6f9636796e0ae54586148ebac0b38b39d4eee7df9e688f05714634f55
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
!pip install -Uq razdel allennlp torch fasttext OpenNMT-py networkx pymorphy2 nltk rouge==0.3.1 summa
!pip install -Uq transformers youtokentome

     |████████████████████████████████| 512kB 18.7MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 204kB 30.6MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 1.4MB 50.1MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 266kB 50.0MB/s 
     |████████████████████████████████| 133kB 53.2MB/s 
     |████████████████████████████████| 1.3MB 57.0MB/s 
     |████████████████████████████████| 317kB 52.9MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 2.5MB 53.1MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 8.2MB 63.4MB/s 
     |████████████████████████████████| 7.0MB 62.0MB/s 
     |████████████████████████████████| 71kB 536kB/s 
     |████████████████████████████████| 2.9MB 63.3MB/s 
     

In [5]:
import random
import pandas as pd

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [6]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

## 1 задание: TextRank (порог: 0.35 BLEU)

TextRank - unsupervised метод для составления кратких выжимок из текста. 
Описание метода:

1. Сплитим текст по предложениям
2. Считаем "похожесть" предложений между собой
3. Строим граф предложений с взвешенными ребрами
4. С помощью алгоритм PageRank получаем наиболее важные предложения, на основе которых делаем summary.

Функция похожести можно сделать и из нейросетевых(или около) моделек: FastText, ELMO и BERT. Выберете один метод, загрузите предобученную модель и с ее помощью для каждого предложениия сделайте sentence embedding. С помощью косинусной меры определяйте похожесть предложений.

Предобученные модели можно взять по [ссылке](http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html).

In [7]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [8]:
!wget https://www.dropbox.com/s/ffyiqeshssn12hz/pytorch_model.bin
!wget https://www.dropbox.com/s/xpktqk8dw72c3zi/vocab.txt
!wget https://www.dropbox.com/s/8bhs4mo4axwaipo/bert_config.json

--2020-12-05 21:11:56--  https://www.dropbox.com/s/ffyiqeshssn12hz/pytorch_model.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ffyiqeshssn12hz/pytorch_model.bin [following]
--2020-12-05 21:11:57--  https://www.dropbox.com/s/raw/ffyiqeshssn12hz/pytorch_model.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucba7a05b5f27b8919b72bff1b75.dl.dropboxusercontent.com/cd/0/inline/BEhTbcbcpI-ekt2Q32K8GxssogyW2Gf4EQHn_Tpt_H_cWUnG3Qm1QBZ0UC9BAMFJH9AyR1buVwy_F3nj0DWcwL323c4HMnom8fOdwz2MyYRHOgtxyyh27Xn6Kktqs_I-ve4/file# [following]
--2020-12-05 21:11:57--  https://ucba7a05b5f27b8919b72bff1b75.dl.dropboxusercontent.com/cd/0/inline/BEhTbcbcpI-ekt2Q32K8GxssogyW2Gf4EQHn_Tpt_H_cWUnG3Qm1QBZ0UC9BAMFJH9AyR1buVwy_F3nj0DWcwL323c4HMnom

In [9]:
import transformers
import torch
class Seq2Vec():
  def __init__(self,vocabName="vocab.txt",bertConfig = "bert_config.json",bertName="pytorch_model.bin", tokenizer_name = "bert-base-multilingual-cased"):
    self.tokenizer = transformers.BertTokenizer.from_pretrained(tokenizer_name)
    self.simple_vocab = {}
    with open(vocabName,"r") as vocab:
      all_text = vocab.read().split('\n')
      for i in range(len(all_text)):
        self.simple_vocab[all_text[i]] = i
    self.model = transformers.BertModel.from_pretrained(bertName, config = bertConfig, output_hidden_states=True)
    #self.model = transformers.BertModel.from_pretrained(tokenizer_name,output_hidden_states=True)
    self.model.to("cuda")
    self.model.eval()
    self.data = {}

    self.PAD = self.simple_vocab['[PAD]']
    self.SEP = self.simple_vocab['[SEP]']
    self.CLS = self.simple_vocab['[CLS]']
    #self.tokenizer.to("cuda")
  def get_ids(self,tokens):
    result = [self.CLS]
    for k in tokens:
      if (k in self.simple_vocab):
         result.append(self.simple_vocab[k])
      else: result.append(0)
    #print(tokens, result)
    result.append(self.SEP)
    if (result == [self.CLS]): return [self.CLS,0,self.SEP]
    return result
  def get_vec(self,sequence):
    with torch.no_grad():
      if("|".join(sequence) in self.data):
        #print("YES")
        return self.data["|".join(sequence)]
      tokens = self.tokenizer.tokenize(" ".join(sequence))
      #tokens = ['[CLS]'] + tokens + ['[SEP]']
      #tokens = self.tokenizer.convert_tokens_to_ids(tokens)
      tokens = self.get_ids(tokens)
      #print(tokens)
      #print(tokens)
      embeddings = self.model(torch.LongTensor([tokens]).to("cuda")).last_hidden_state[:,0,:]
      
      #print(embeddings.shape)
      res = embeddings.cpu().reshape(-1).detach().numpy()
      #print(res.shape)
      self.data["|".join(sequence)] = res
      return res
#path = 'navec_news_v1_1B_250K_300d_100q.tar'
#model = navec
model = Seq2Vec()
#from navec import Navec
#path = 'navec_news_v1_1B_250K_300d_100q.tar'
#navec = Navec.load(path) 

In [10]:
from itertools import combinations
import networkx as nx
import numpy as np
import pymorphy2
import razdel
from numpy.linalg import norm
import tensorflow as tf
import tensorflow_hub as hub
#import transformers
#model = transformers.BertTokenizer.from_pretrained("RuBERT.gz", cache_dir=None)
#print(model)
ITERS = 0
def your_super_words_similarity(words1, words2, embeddings = model):
    #TEMP
    global ITERS
    ITERS+=1
    v1, v2 = model.get_vec(words1), model.get_vec(words2)
    v1 = v1/np.linalg.norm(v1)
    v2 = v2/np.linalg.norm(v2)
    if (ITERS%100000==0): print(ITERS)
    #print(v1.dot(v2))
    return v1.dot(v2)


def gen_text_rank_summary(text, calc_similarity=your_super_words_similarity, summary_part=0.1, lower=True, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    # Токенизируем предложения
    sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]

    # При необходимости лемматизируем слова
    if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2) 
    scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    pr = nx.pagerank_numpy(g)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])

    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    return predicted_summary

def calc_text_rank_score(records, calc_similarity=your_super_words_similarity, summary_part=0.1, lower=True, nrows=1000, morph=None):
    references = []
    predictions = []

    for text, summary in records[['text', 'summary']].values[:nrows]:
        summary = summary if not lower else summary.lower()
        references.append(summary)

        predicted_summary = gen_text_rank_summary(text, calc_similarity, summary_part, lower, morph=morph)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)
    
    calc_scores(references, predictions)

In [14]:
calc_text_rank_score(test_records, calc_similarity=your_super_words_similarity)

100000
200000
300000
400000
500000
600000
700000
Count: 1000
Ref: поскольку сша и европа первыми ввели санкции против россии, то наша страна не станет в одностороннем порядке отменять их. и хотя с 2014 года, когда были введены первые санкции, в сельском хозяйстве появились новые товары отечественного производства, из-за девальвации рубля продукты сильно подорожали.
Hyp: ключевым торговым партнером россии по-прежнему остается евросоюз. часть европейских поставок все равно достигают россии, но они стали осуществляться через минск. по его словам, геополитическая ситуация пока не позволяет это предпринять.
BLEU:  0.35574776662378244
ROUGE:  {'rouge-1': {'f': 0.14135560669836073, 'p': 0.144132094192153, 'r': 0.1463283864337989}, 'rouge-2': {'f': 0.025196085720647264, 'p': 0.02597286647977443, 'r': 0.025906311548126434}, 'rouge-l': {'f': 0.12320597926503388, 'p': 0.13125431533009035, 'r': 0.13292647573332}}


## 2 Задание: Extractive RNN (порог: 0.35 BLEU)

Второй метод, который вам предлагается улучшить – поиск предложений для summary с помощью RNN. В рассмотренной методе мы использовали LSTM для генерации sentence embedding. Попробуйте использовать другие архитектуры: CNN, Transformer; или добавьте предобученные модели, как и в первом задании.

P.S. Тут предполагается, что придется изменять много кода в ячееках (например, поменять токенизацию). 

### Модель

Картинка для привлечения внимания:

![img](https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_14%2Fproject_398421%2Fimages%2Farchitecture.png)

Статья с оригинальным методом:
https://arxiv.org/pdf/1611.04230.pdf

Список вдохновения: 
- https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b Пример того, как можно применять CNN в текстовых задачах
- https://arxiv.org/pdf/1808.08745.pdf Очень крутой метод генерации summary без Transformers
- https://towardsdatascience.com/super-easy-way-to-get-sentence-embedding-using-fasttext-in-python-a70f34ac5b7c – простой метод генерации sentence embedding
- https://towardsdatascience.com/fse-2b1ffa791cf9 – Необычный метод генерации sentence embedding
- https://github.com/UKPLab/sentence-transformers – BERT предобученный для sentence embedding

P.S. Выше написанные ссылки нужны только для разогрева вашей фантазии, можно воспользоваться ими, а можно придумать свой.

Комментарий к заданию:
Если посмотреть на архитектуру ~~почти~~ SummaRuNNer, то в ней есть два главных элемента: первая часть, которая читает предложения и возвращает векторы на каждое предложение, и вторая, которая выбирает предложения для суммаризации. Вторую часть мы не трогаем, а первую меняем. На что меняем – как вы решите. Главное: она должна иметь хорошее качество и встроиться в текущую модель.

In [11]:
import copy
import random

def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    score = -1.0
    summaries = []
    for _ in range(min(n_sentences, 2)):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [12]:
from tqdm import tqdm_notebook as tqdm

def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for text, summary in tqdm(records[['text', 'summary']].values[:nrows]):
        summary = summary if not lower else summary.lower()
        references.append(summary)
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_oracle_score(test_records)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Count: 1000
Ref: телеведущий дмитрий шепелев рассказал о непростых отношениях с семьей своей покойной возлюбленной жанны фриске. по словам шоумена, суд постановил родителям певицы видеться с внуком по полтора часа два раза в месяц, но они ни разу к нему не приехали.
Hyp: ведущий передачи «на самом деле» дмитрий шепелев рассказал об отношениях с семьей скончавшейся от рака певицы жанны фриске. он подчеркнул, что, согласно постановлению суда, родители фриске могут видеться с внуком по полтора часа два раза в месяц.
BLEU:  0.42589664646845066
ROUGE:  {'rouge-1': {'f': 0.3490451460584015, 'p': 0.42940880325806274, 'r': 0.3121174553668983}, 'rouge-2': {'f': 0.19940504508059612, 'p': 0.2527668794480655, 'r': 0.17716857908080433}, 'rouge-l': {'f': 0.29934515952910823, 'p': 0.4004867192385652, 'r': 0.2894968565537724}}


## (!)
Если надо, поменяйте код загрузки токенизатора

In [13]:
import os

import youtokentome as yttm

def train_bpe(records, model_path, model_type="bpe", vocab_size=30000, lower=True):
    temp_file_name = "temp.txt"
    with open(temp_file_name, "w") as temp:
        for text, summary in records[['text', 'summary']].values:
            if lower:
                summary = summary.lower()
                text = text.lower()
            if not text or not summary:
                continue
            temp.write(text + "\n")
            temp.write(summary + "\n")
    yttm.BPE.train(data=temp_file_name, vocab_size=vocab_size, model=model_path)

train_bpe(train_records, "BPE_model.bin")

In [14]:
bpe_processor = yttm.BPE('BPE_model.bin')
bpe_processor.encode(["октябрь богат на изменения"], output_type=yttm.OutputType.SUBWORD)

[['▁октябрь', '▁богат', '▁на', '▁изменения']]

## (!)
Если надо, поменяйте код словаря

In [15]:
from collections import Counter
from typing import List, Tuple
import os

class Vocabulary:
    def __init__(self, bpe_processor):
        self.index2word = bpe_processor.vocab()
        self.word2index = {w: i for i, w in enumerate(self.index2word)}
        self.word2count = Counter()

    def get_pad(self):
        return self.word2index["<PAD>"]

    def get_sos(self):
        return self.word2index["<SOS>"]

    def get_eos(self):
        return self.word2index["<EOS>"]

    def get_unk(self):
        return self.word2index["<UNK>"]
    
    def has_word(self, word) -> bool:
        return word in self.word2index

    def get_index(self, word):
        if word in self.word2index:
            return self.word2index[word]
        return self.get_unk()

    def get_word(self, index):
        return self.index2word[index]

    def size(self):
        return len(self.index2word)

    def is_empty(self):
        empty_size = 4
        return self.size() <= empty_size

    def reset(self):
        self.word2count = Counter()
        self.index2word = ["<pad>", "<sos>", "<eos>", "<unk>"]
        self.word2index = {word: index for index, word in enumerate(self.index2word)}

In [16]:
vocabulary = Vocabulary(bpe_processor)
vocabulary.size()

30000

In [17]:
from rouge import Rouge
import razdel

def add_oracle_summary_to_records(records, max_sentences=30, lower=True, nrows=1000):
    rouge = Rouge()
    sentences_ = []
    oracle_sentences_ = []
    oracle_summary_ = []
    if nrows is not None:
        records = records.iloc[:nrows].copy()
    else:
        records = records.copy()

    for text, summary in tqdm(records[['text', 'summary']].values):
        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge),
                                                                         lower=lower, max_sentences=max_sentences)
        sentences_ += [sentences]
        oracle_sentences_ += [list(sentences_indicies)]
        oracle_summary_ += [oracle_summary]
    records['sentences'] = sentences_
    records['oracle_sentences'] = oracle_sentences_
    records['oracle_summary'] = oracle_summary_
    return records

# ext_train_records = add_oracle_summary_to_records(train_records, nrows=30000)
# ext_val_records = add_oracle_summary_to_records(val_records, nrows=None)
# ext_test_records = add_oracle_summary_to_records(test_records, nrows=None)

Используй `pickle` для сохранения записей, чтобы потом не пересоздавать их потом. Если решаешь задание в колабе, можешь подключить свой гугл диск и сохранить данные в нём.

In [18]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

pickle_train = open("/content/drive/MyDrive/train_records.bin","rb")
pickle_test = open("/content/drive/MyDrive/test_records.bin","rb")
pickle_valid = open("/content/drive/MyDrive/val_records.bin","rb")

ext_test_records  = pickle.load(pickle_test)
ext_val_records = pickle.load(pickle_valid)
ext_train_records = pickle.load(pickle_train)

Mounted at /content/drive


In [19]:
# import pickle
# from google.colab import drive
# drive.mount('/content/drive')

# with open("train_records.bin", 'wb') as file:
#     pickle.save(file, ext_train_records)

## (!)
Если надо, поменяйте код генератора датасета и батчевалки

In [20]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


from torch.utils import data


class ExtDataset(data.Dataset):
    def __init__(self, records, vocabulary, bpe_processor, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = records.shape[0]
        self.bpe_processor = bpe_processor
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.records.shape[0]

    def __getitem__(self, idx):
        cur_record = self.records.iloc[idx]
        inputs = list(map(lambda x: x[:self.max_sentence_length], self.bpe_processor.encode(cur_record['sentences'], output_type=yttm.OutputType.ID)))
        outputs = [int(i in cur_record['oracle_sentences']) for i in range(len(cur_record['sentences']))]
        return {'inputs': inputs, 'outputs': outputs}

In [21]:
# Это батчевалка
def collate_fn(records):
    max_length = max(len(sentence) for record in records for sentence in record['inputs'])
    max_sentences = max(len(record['outputs']) for record in records)

    new_inputs = torch.zeros((len(records), max_sentences, max_length))
    new_outputs = torch.zeros((len(records), max_sentences))
    for i, record in enumerate(records):
        for j, sentence in enumerate(record['inputs']):
            new_inputs[i, j, :len(sentence)] += np.array(sentence)
        new_outputs[i, :len(record['outputs'])] += np.array(record['outputs'])
    return {'features': new_inputs.type(torch.LongTensor), 'targets': new_outputs}

In [27]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super().__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Embedding(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)

        return sentences_embeddings

class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        
        super().__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = SentenceEncoderRNN(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers, 
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional)
        
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()

    def forward(self, inputs, hidden=None):

        # [batch_size, seq num, seq_len]
        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count)
        # [batch_size * seq num, seq_len]

        embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        # [batch_size, seq num, hidden_size]

        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        # [batch_size, seq num, hidden_size]

        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        # [batch_size, hidden_size]

        # W * h^T
        content = self.content_linear_layer(outputs).squeeze(2) # 1-representation
        # [batch_size, seq num]

        # h^T * W * d
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2) # 2-representation

        # [batch_size, seq num, hidden_size] * [batch_size, hidden_size, 1] = [batch_size, seq num, 1
        return content + salience

model = SentenceTaggerRNN(vocabulary.size())

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


### Обучение

In [28]:
device = torch.device('cuda')

loaders = {
    'train': data.DataLoader(
        ExtDataset(
            ext_train_records, 
            vocabulary, 
            bpe_processor=bpe_processor
        ), 
        batch_size=64, 
        collate_fn=collate_fn
    ),
    'valid': data.DataLoader(
        ExtDataset(
            ext_val_records, 
            vocabulary, 
            bpe_processor=bpe_processor
        ), 
        batch_size=64, 
        collate_fn=collate_fn
    ),
    'test': data.DataLoader(
        ExtDataset(
            ext_test_records, 
            vocabulary, 
            bpe_processor=bpe_processor
        ), 
        batch_size=1, 
        collate_fn=collate_fn
    ),
}

lr = 1e-4
num_epochs = 1

optimizer  = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()
# Maybe adding scheduler?

In [29]:
from tqdm.notebook import trange, tqdm


def train():
    model.to(device)
    pbar_loader = trange(len(loaders["train"]) + len(loaders["valid"]), desc=f"Train Loss: {0}, Valid Loss: {0}")
    for e in trange(num_epochs, desc="Epoch"):
        train_loss = 0
        valid_loss = 0
        train_it = 0
        valid_it = 0
        
        model.train()
        for batch in loaders["train"]:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)
            
            logits = model(features)
            
            loss = criterion(logits, targets)
            train_loss += loss.item()
            train_it += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Maybe adding scheduler?
            
            pbar_loader.update()
            pbar_loader.set_description(
                f"Train Loss: {train_loss / train_it:.3}, Valid Loss: {0}"
            )
            
        model.eval()
        with torch.no_grad():
            for batch in loaders["valid"]:
                features = batch["features"].to(device)
                targets = batch["targets"].to(device)

                logits = model(features)

                loss = criterion(logits, targets)
                valid_loss += loss.item()
                valid_it += 1
                
                pbar_loader.update()
                pbar_loader.set_description(
                    f"Train Loss: {train_loss / train_it:.3},"
                    f" Valid Loss: {valid_loss / valid_it:.3}"
                )
        print(
            f"Epoch {e}; Train Loss: {train_loss / train_it:.3},"
            f" Valid Loss: {valid_loss / valid_it:.3}"
        )
        pbar_loader.reset()

In [30]:
train()

Epoch 0; Train Loss: 0.204, Valid Loss: 0.182



In [32]:
device = torch.device("cuda")

top_k = 3
references = []
predictions = []

def postprocess(ref, hyp, is_multiple_ref=False, detokenize_after=False, tokenize_after=True):
    if is_multiple_ref:
        reference_sents = ref.split(" s_s ")
        decoded_sents = hyp.split("s_s")
        hyp = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in decoded_sents]
        ref = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in reference_sents]
        hyp = " ".join(hyp)
        ref = " ".join(ref)
    ref = ref.strip()
    hyp = hyp.strip()
    if detokenize_after:
        hyp = punct_detokenize(hyp)
        ref = punct_detokenize(ref)
    if tokenize_after:
        hyp = hyp.replace("@@UNKNOWN@@", "<unk>")
        hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
        ref = " ".join([token.text for token in razdel.tokenize(ref)])
    return ref, hyp

model.eval()
for num, batch in enumerate(loaders["test"]):

    logits = model(batch["features"].to(device))
    in_summary = torch.argsort(logits, dim=1)[:, -top_k:]
    for i in range(len(batch['targets'])):

        summary = ext_test_records.iloc[i]['summary']
        summary = summary.lower()
        predicted_summary = ' '.join([ext_test_records.iloc[i]['sentences'][idx] for idx in in_summary[i].sort()[0] if idx < len(ext_test_records.iloc[i]['sentences'])])
        summary, predicted_summary = postprocess(summary, predicted_summary)

        references.append(summary)
        predictions.append(predicted_summary)

calc_scores(references, predictions)

Count: 5770
Ref: в акваторию черного моря зашел американский эсминец ross , у которого на вооружении имеются ракеты и средства пво . вместе с тем неизвестно , куда он следует . пока он находится в черноморской акватории , его сопровождает российское судно « вышний волочек » .
Hyp: в юго-западную часть акватории черного моря вошел американский эсминец ross . об этом сообщили в национальном центре управления обороной , передает риа « новости » . там отметили , что корабль идет в сопровождении российского судна черноморского флота « вышний волочек » .
BLEU:  0.5548088998026359
ROUGE:  {'rouge-1': {'f': 0.33333332833717866, 'p': 0.34285714285715496, 'r': 0.3243243243243641}, 'rouge-2': {'f': 0.15189872917801497, 'p': 0.15384615384616374, 'r': 0.14999999999998517}, 'rouge-l': {'f': 0.3328201180390804, 'p': 0.34285714285715496, 'r': 0.3243243243243641}}
